# API (參考https://github.com/mlouielu/twstock)

In [35]:
import datetime
import urllib.parse
from collections import namedtuple
import time
import random
import requests
from codes import codes

#from twstock.proxy import get_proxies

try:
    from json.decoder import JSONDecodeError
except ImportError:
    JSONDecodeError = ValueError


"""
try:
    from . import analytics
    from .codes import codes
except ImportError as e:
    if e.name == 'lxml':
        # Fix #69
        raise e
    import analytics
    from codes import codes
"""

headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/537.36'}


TWSE_BASE_URL = 'http://www.twse.com.tw/'
TPEX_BASE_URL = 'http://www.tpex.org.tw/'
DATATUPLE = namedtuple('Data', ['date', 'capacity', 'turnover', 'open',
                                'high', 'low', 'close', 'change', 'transaction'])


class BaseFetcher(object):
    def fetch(self, year, month, sid, retry):
        pass

    def _convert_date(self, date):
        """Convert '106/05/01' to '2017/05/01'"""
        return '/'.join([str(int(date.split('/')[0]) + 1911)] + date.split('/')[1:])

    def _make_datatuple(self, data):
        pass

    def purify(self, original_data):
        pass


class TWSEFetcher(BaseFetcher):
    REPORT_URL = urllib.parse.urljoin(
        TWSE_BASE_URL, 'exchangeReport/STOCK_DAY')

    def __init__(self):
        pass

    def fetch(self, year: int, month: int, sid: str, retry: int=5):
        params = {'date': '%d%02d01' % (year, month), 'stockNo': sid}
        for retry_i in range(retry):
            r = requests.get(self.REPORT_URL, params=params, headers=headers)
            try:
                data = r.json()
                #print(data)
            except JSONDecodeError:
                print('JSONDecodeError:{}, retry={}'.format(sid, retry_i))
                continue
            else:
                break
        else:
            # Fail in all retries
            data = {'stat': '', 'data': []}

        if data['stat'] == 'OK':
            data['data'] = self.purify(data)
        else:
            data['data'] = []
        time.sleep(random.uniform(2,5))
        return data

    def _make_datatuple(self, data):
        data[0] = datetime.datetime.strptime(
            self._convert_date(data[0]), '%Y/%m/%d')
        data[1] = int(data[1].replace(',', ''))
        data[2] = int(data[2].replace(',', ''))
        data[3] = None if data[3] == '--' else float(data[3].replace(',', ''))
        data[4] = None if data[4] == '--' else float(data[4].replace(',', ''))
        data[5] = None if data[5] == '--' else float(data[5].replace(',', ''))
        data[6] = None if data[6] == '--' else float(data[6].replace(',', ''))
        # +/-/X表示漲/跌/不比價
        data[7] = float(0.0 if data[7].replace(',', '') ==
                        'X0.00' else data[7].replace(',', ''))
        data[8] = int(data[8].replace(',', ''))
        return DATATUPLE(*data)

    def purify(self, original_data):
        return [self._make_datatuple(d) for d in original_data['data']]



class TPEXFetcher(BaseFetcher):
    REPORT_URL = urllib.parse.urljoin(TPEX_BASE_URL,
                                      'web/stock/aftertrading/daily_trading_info/st43_result.php')

    def __init__(self):
        pass

    def fetch(self, year: int, month: int, sid: str, retry: int=5):
        params = {'d': '%d/%d' % (year - 1911, month), 'stkno': sid}
        for retry_i in range(retry):
            r = requests.get(self.REPORT_URL, params=params)
            try:
                data = r.json()
            except JSONDecodeError:
                time.sleep(2)
                continue
            else:
                break
        else:
            # Fail in all retries
            data = {'aaData': []}

        data['data'] = []
        if data['aaData']:
            data['data'] = self.purify(data)
        
        return data

    def _convert_date(self, date):
        """Convert '106/05/01' to '2017/05/01'"""
        return '/'.join([str(int(date.split('/')[0]) + 1911)] + date.split('/')[1:])

    def _make_datatuple(self, data):
        data[0] = datetime.datetime.strptime(self._convert_date(data[0].replace('＊', '')),
                                             '%Y/%m/%d')
        data[1] = int(data[1].replace(',', '')) * 1000
        data[2] = int(data[2].replace(',', '')) * 1000
        data[3] = None if data[3] == '--' else float(data[3].replace(',', ''))
        data[4] = None if data[4] == '--' else float(data[4].replace(',', ''))
        data[5] = None if data[5] == '--' else float(data[5].replace(',', ''))
        data[6] = None if data[6] == '--' else float(data[6].replace(',', ''))
        data[7] = float(data[7].replace(',', ''))
        data[8] = int(data[8].replace(',', ''))
        return DATATUPLE(*data)

    def purify(self, original_data):
        return [self._make_datatuple(d) for d in original_data['aaData']]


class Stock(sid):

    def __init__(self, sid: str, initial_fetch: bool=True):
        self.sid = sid
        self.fetcher = TWSEFetcher(
        ) if codes[sid].market == '上市' else TPEXFetcher()
        self.raw_data = []
        self.data = []

        # Init data
        if initial_fetch:
            self.fetch_31()

    def _month_year_iter(self, start_month, start_year, end_month, end_year):
        ym_start = 12 * start_year + start_month - 1
        ym_end = 12 * end_year + end_month
        for ym in range(ym_start, ym_end):
            y, m = divmod(ym, 12)
            yield y, m + 1

    def fetch(self, year: int, month: int):
        """Fetch year month data"""
        self.raw_data = [self.fetcher.fetch(year, month, self.sid)]
        self.data = self.raw_data[0]['data']
        return self.data

    def fetch_from(self, year: int, month: int):
        """Fetch data from year, month to current year month data"""
        self.raw_data = []
        self.data = []
        today = datetime.datetime.today()
        for year, month in self._month_year_iter(month, year, today.month, today.year):
            self.raw_data.append(self.fetcher.fetch(year, month, self.sid))
            self.data.extend(self.raw_data[-1]['data'])
        return self.data

    def fetch_31(self):
        """Fetch 31 days data"""
        today = datetime.datetime.today()
        before = today - datetime.timedelta(days=60)
        self.fetch_from(before.year, before.month)
        self.data = self.data[-31:]
        return self.data

    @property
    def date(self):
        return [d.date for d in self.data]

    @property
    def capacity(self):
        return [d.capacity for d in self.data]

    @property
    def turnover(self):
        return [d.turnover for d in self.data]

    @property
    def price(self):
        return [d.close for d in self.data]

    @property
    def high(self):
        return [d.high for d in self.data]

    @property
    def low(self):
        return [d.low for d in self.data]

    @property
    def open(self):
        return [d.open for d in self.data]

    @property
    def close(self):
        return [d.close for d in self.data]

    @property
    def change(self):
        return [d.change for d in self.data]

    @property
    def transaction(self):
        return [d.transaction for d in self.data]

# 取得某年月股價 (fetch)

In [36]:
stock=Stock('2330')
stock.fetch(2020,5)

[Data(date=datetime.datetime(2020, 5, 4, 0, 0), capacity=72252861, turnover=21341931218, open=294.5, high=296.5, low=294.0, close=295.0, change=-9.5, transaction=37736),
 Data(date=datetime.datetime(2020, 5, 5, 0, 0), capacity=23988405, turnover=7104774785, open=296.5, high=298.0, low=295.0, close=295.5, change=0.5, transaction=13569),
 Data(date=datetime.datetime(2020, 5, 6, 0, 0), capacity=35704479, turnover=10524838077, open=294.5, high=296.0, low=292.5, close=296.0, change=0.5, transaction=16046),
 Data(date=datetime.datetime(2020, 5, 7, 0, 0), capacity=28017198, turnover=8321170642, open=294.5, high=299.0, low=294.5, close=297.5, change=1.5, transaction=13149),
 Data(date=datetime.datetime(2020, 5, 8, 0, 0), capacity=35613893, turnover=10619091212, open=300.0, high=300.0, low=296.0, close=297.5, change=0.0, transaction=14345),
 Data(date=datetime.datetime(2020, 5, 11, 0, 0), capacity=32379019, turnover=9729431840, open=300.0, high=301.5, low=298.5, close=301.0, change=3.5, transac

# 取得最近31天股價

In [60]:
#最近31天
stock=Stock('2330')
stock.fetch_31
stock.data

[Data(date=datetime.datetime(2020, 4, 6, 0, 0), capacity=59712754, turnover=16324198154, open=273.0, high=275.5, low=270.0, close=275.5, change=4.0, transaction=19971),
 Data(date=datetime.datetime(2020, 4, 7, 0, 0), capacity=48887346, turnover=13817936851, open=283.5, high=284.0, low=280.5, close=283.0, change=7.5, transaction=24281),
 Data(date=datetime.datetime(2020, 4, 8, 0, 0), capacity=38698826, turnover=11016972354, open=285.0, high=285.5, low=283.0, close=285.0, change=2.0, transaction=19126),
 Data(date=datetime.datetime(2020, 4, 9, 0, 0), capacity=29276430, turnover=8346209654, open=287.5, high=288.0, low=282.5, close=283.0, change=-2.0, transaction=15271),
 Data(date=datetime.datetime(2020, 4, 10, 0, 0), capacity=28206858, turnover=7894277586, open=280.0, high=282.0, low=279.0, close=279.5, change=-3.5, transaction=15833),
 Data(date=datetime.datetime(2020, 4, 13, 0, 0), capacity=22196596, turnover=6202532380, open=278.5, high=281.5, low=278.5, close=278.5, change=-1.0, tran

# 列印出日期，股價, 成交量

In [57]:
for _date, _price, _turnover in zip(stock.date, stock.price, stock.turnover):
    print(_date, _price, _turnover)

2020-04-06 00:00:00 275.5 16324198154
2020-04-07 00:00:00 283.0 13817936851
2020-04-08 00:00:00 285.0 11016972354
2020-04-09 00:00:00 283.0 8346209654
2020-04-10 00:00:00 279.5 7894277586
2020-04-13 00:00:00 278.5 6202532380
2020-04-14 00:00:00 285.0 14268689661
2020-04-15 00:00:00 287.5 16819851630
2020-04-16 00:00:00 286.5 13813721932
2020-04-17 00:00:00 306.5 35719172728
2020-04-20 00:00:00 304.0 13605802989
2020-04-21 00:00:00 295.0 18417786171
2020-04-22 00:00:00 294.0 12746170556
2020-04-23 00:00:00 295.5 11864085990
2020-04-24 00:00:00 294.0 7799075756
2020-04-27 00:00:00 298.0 11461378300
2020-04-28 00:00:00 296.5 11134919970
2020-04-29 00:00:00 299.0 13861245296
2020-04-30 00:00:00 304.5 17753466901
2020-05-04 00:00:00 295.0 21341931218
2020-05-05 00:00:00 295.5 7104774785
2020-05-06 00:00:00 296.0 10524838077
2020-05-07 00:00:00 297.5 8321170642
2020-05-08 00:00:00 297.5 10619091212
2020-05-11 00:00:00 301.0 9729431840
2020-05-12 00:00:00 295.0 15553012596
2020-05-13 00:00:00